In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import seaborn as sns

from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import os
from pathlib import Path
import sys
import glob
import re

In [ ]:
# Get the directory of the running notebook
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent  # Adjust based on your project structure

sys.path.append(str(project_root))

In [ ]:
# konstante Variablen
from Variables.constants import *

In [ ]:
from Importfunctions.import_specs import * # allgemeine Datensatzverarbeitungsfunktionen
from Altersgruppierung.age_grouping import *
from Postprocessing.filter import * # Frequenzfilter
from Postprocessing.data_sanitizing import * # Formatierung und Anderes
from Postprocessing.diag_proc import * #häufigste, individuelle Diagnosen
from Farbtonerstellung.hues import *
from Darstellungen.plotting import *
from Darstellungen.axis_config import * # konfiguriere die Achsen je nach Position im Plot

In [ ]:
# erhöhe die Seaborn-Auflösung
sns.set_theme(rc={"figure.dpi": 600})

# Funktion für die Spektrendarstellung

In [ ]:
'''
Die Referenzspektren-Dataframes ref_spec_df und ref_chemshift_df sind bezogen auf die Echozeit und nicht auf
die einzelnen Feldstärken
'''
def farbcodierte_darstellung(df_befund,df_spec,df_chemshift,diag,ref_spec_df,ref_chemshift_df):

    index_temp = spec_selec(df_befund,df_spec,diag=diag)

    # entferne alle Voxelpositionen bis auf Marklager und Basalganglien
    index_temp = entferne_irr_vp(df_befund,index_temp)

    # wähle die eingeschlossenen Altersgruppen aus
    alter_gr_selec = waehle_altersgruppen(df_befund,index_temp)

    # Bestimme wieviel Zeilen man braucht
    n_altersgruppen = len(alter_gr_selec)
    row_n = (np.floor(n_altersgruppen/2)+1).astype(int)

    # initialisiere Abbildungen und gebe Titel
    fig = plt.figure(figsize=(10, (10/3)*row_n))
    fig.suptitle(diag)

    outer = gridspec.GridSpec(row_n, 2, wspace=0.2, hspace=.25)

    for i, alter_selec in enumerate(alter_gr_selec):  # bis zur letzten Altersgruppe
        inner = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[i], wspace=0.1, hspace=.1)

        for j in range(2):
            ax = plt.Subplot(fig, inner[j])
            vp_selec = VP_NAMES[j]
            index_temp = spec_selec(df_befund,df_spec,alter_categ=alter_selec,diag=diag,vp=vp_selec)

            #Konfiguriere die Achse entsprechend der Stellung in der Abbildung
            ax = axis_config_einzeldiag(j,i,ax,vp_selec,index_temp)

            # Farbtonauswahl
            palette = select_palette(None)

            if not index_temp.empty:
                spec_proc = process_specs(df_befund, df_spec,df_chemshift,index_temp)
                ax = fig.add_subplot(ax)

                sns.lineplot(data=spec_proc, ax=ax, x='x', y='y', linewidth=.5, palette=palette,
                    estimator=None, units='index',hue='Farbton',hue_norm=(0,1), legend=None)
                
                if (vp_selec,alter_selec) in ref_spec_df.index.to_list():
                    sns.lineplot(x=ref_chemshift_df.loc[vp_selec,alter_selec],y=ref_spec_df.loc[vp_selec,alter_selec],
                                linewidth=1, color='black', alpha = 0.5,
                                label='Referenzspektrum')
                    ax.legend(loc=(.7,.75),fontsize=6,frameon=False)
            else:
                fig.add_subplot(ax)
                
            create_colorbar(alter_selec, palette, ax, interval_n=8)

    # Füge jetzt noch die Darstellungen mit der Altersverteilung hinzu
    i = i+1
    if np.mod(i,2)==1:
        inner = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[i], wspace=0.1, hspace=.1)
    else:
        inner = gridspec.GridSpecFromSubplotSpec(2, 3,
                    subplot_spec=outer[i:], width_ratios=[.25,.5,.25])
        inner = gridspec.GridSpecFromSubplotSpec(2,1,subplot_spec=inner[0:,1],wspace=0.1, hspace=0.1)


    for j in range(2):
        ax = plt.Subplot(fig, inner[j])
        ax = fig.add_subplot(ax)
        vp_selec = VP_NAMES[j]
        index_temp = spec_selec(df_befund,df_spec,diag=diag,vp=vp_selec)

        if not index_temp.empty:
            data = df_befund['Alter']['Jahre'].loc[index_temp]
            plot_distribution(data,title='',ax=ax)

        ax = axis_config_histo(j,i,vp_selec,ax,index_temp)

    return fig

# Lade alle Spektren aus der Pickle-Datei

In [ ]:
os.chdir("path/to/pickled/data")
with open('df_pickled.pickle','rb') as f:
    ds = pickle.load(f)

# wähle einen Datensatz aus

In [ ]:
i = 1
df_befund, df_spec, df_chemshift, ref_spec_df, ref_chemshift_df = ds.load_dataset(i)

df_name = FILE_NAMES[i-1].strip('*') #wegen Zero-Indexing

# Häufigste Einzeldiagnosen

In [ ]:
einzeldiag_selec_indices = häufigste_befunde_diag(df_befund,df_spec)

In [ ]:
einzeldiag_list = list(einzeldiag_selec_indices.keys())

In [ ]:
einzeldiag_list

# Farbkodierte Darstellung

In [ ]:
# Füge die Spalte mit den Farbtönen hinzu
determine_hues(df_befund)

# Speichern der Darstellungen

In [ ]:
os.chdir("path/to/save/graphs")
for diag_selec in einzeldiag_list:
    fig = farbcodierte_darstellung(df_befund,df_spec,df_chemshift,
                                   diag=diag_selec,ref_chemshift_df=ref_chemshift_df,
                                   ref_spec_df=ref_spec_df)
    fig.savefig(diag_selec+f'_{df_name}_farbkodiert.jpeg')